# <center>Lab03 Transformers</center>

## HuggingFace Transformers

Let's import the development version of the HuggingFace Transformer

In [1]:
!pip install transformers[sentencepiece]

/bin/bash: /home/pili/anaconda3/envs/scia/lib/libtinfo.so.6: no version information available (required by /bin/bash)


## Sentence classification

Using HuggingFace transformer library to fine-tune a model on the IMDB library dataset and then evaluating it on the test set.

In [12]:
import transformers
from datasets import load_dataset
import numpy as np

from IPython.display import Markdown, display

BATCH_SIZE = 10

# we will use the distilbert model$

In [3]:
# working with the GPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
dataset = load_dataset("imdb")
train_dataset = dataset["train"].train_test_split(
    stratify_by_column="label", test_size=0.2, seed=42
)
test_df = dataset["test"]
train_df = train_dataset["train"]
valid_df = train_dataset["test"]
train_df.shape, valid_df.shape, test_df.shape

Found cached dataset imdb (/home/pili/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 1003.02it/s]
Loading cached split indices for dataset at /home/pili/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-5f37fd0866e4f89f.arrow and /home/pili/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-dd5732a0e6ac784c.arrow


((20000, 2), (5000, 2), (25000, 2))

### Preprocessing

In [5]:
# preprocessing
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format("torch")

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /home/pili/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-74e187f956cd9276.arrow
Loading cached processed dataset at /home/pili/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-bf12997ecf0c40f9.arrow
2023-05-30 16:23:27.353781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 16:23:28.190242: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Fine tuning of the model using the accuracy

Training args used, the max batch size we could use was 10, it means 2000 steps for a dataset of 20 000 images.  
Every 100 steps we output a logging and every 500 steps we display the accuracy on the validation set and we save the model

In [6]:
# training
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    
)

We use a pretrained model `distilbert-base-uncased`

In [7]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classi

Implementing custom metrics using accuracy :

In [8]:
import evaluate

def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Getting trainer parameters then training begins :

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

/home/pili/anaconda3/envs/scia/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  5%|▌         | 100/2000 [00:42<13:12,  2.40it/s]

{'loss': 0.4768, 'learning_rate': 4.75e-05, 'epoch': 0.05}


 10%|█         | 200/2000 [01:25<12:42,  2.36it/s]

{'loss': 0.364, 'learning_rate': 4.5e-05, 'epoch': 0.1}


 15%|█▌        | 300/2000 [02:07<11:53,  2.38it/s]

...

                                                   
100%|██████████| 2000/2000 [19:14<00:00,  2.36it/s]

{'eval_loss': 0.23784467577934265, 'eval_accuracy': 0.9162, 'eval_runtime': 75.9594, 'eval_samples_per_second': 65.825, 'eval_steps_per_second': 6.582, 'epoch': 1.0}


100%|██████████| 2000/2000 [19:15<00:00,  1.73it/s]

{'train_runtime': 1155.7921, 'train_samples_per_second': 17.304, 'train_steps_per_second': 1.73, 'train_loss': 0.294110538482666, 'epoch': 1.0}


TrainOutput(global_step=2000, training_loss=0.294110538482666, metrics={'train_runtime': 1155.7921, 'train_samples_per_second': 17.304, 'train_steps_per_second': 1.73, 'train_loss': 0.294110538482666, 'epoch': 1.0})

Saving and loading the model to not retrain in case of restart :

In [16]:
# save the model
trainer.save_model("./model")

In [10]:
# load the model
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="./model", tokenizer=checkpoint)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


### Evaluation of the model on the test set

In [20]:
# evaluate the model on the test set
tokenized_test = dataset["test"].map(tokenize_function, batched=True)
tokenized_test.set_format("torch")
trainer.evaluate(tokenized_test)

100%|██████████| 2500/2500 [06:12<00:00,  6.71it/s]                


{'eval_loss': 0.21499311923980713,
 'eval_accuracy': 0.92404,
 'eval_runtime': 372.9686,
 'eval_samples_per_second': 67.03,
 'eval_steps_per_second': 6.703,
 'epoch': 1.0}

We get an accuracy of 0.92404 on the test set wich was expected since we had a 0.9162 score on the evaluation set during training.

### Analysis on wrongly classified examples

In [13]:
nb = 2
for i in range(25000):
    if int(classifier(dataset["test"][i]["text"])[0]["label"][-1]) != int(dataset["test"][i]["label"]):
        display(Markdown(dataset["test"][i]["text"]))
        print("classifier prediction : ", classifier(dataset["test"][i]["text"])[0]["label"][-1])
        print("dataset label : ", dataset["test"][i]["label"])
        nb -= 1
        if nb == 0:
         break

I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn't match the background, and painfully one-dimensional characters cannot be overcome with a 'sci-fi' setting. (I'm sure there are those of you out there who think Babylon 5 is good sci-fi TV. It's not. It's clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It's really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it's rubbish as they have to always say "Gene Roddenberry's Earth..." otherwise people would not continue watching. Roddenberry's ashes must be turning in their orbit as this dull, cheap, poorly edited (watching it without advert breaks really brings this home) trudging Trabant of a show lumbers into space. Spoiler. So, kill off a main character. And then bring him back as another actor. Jeeez! Dallas all over again.

classifier prediction :  1
dataset label :  0


Worth the entertainment value of a rental, especially if you like action movies. This one features the usual car chases, fights with the great Van Damme kick style, shooting battles with the 40 shell load shotgun, and even terrorist style bombs. All of this is entertaining and competently handled but there is nothing that really blows you away if you've seen your share before.<br /><br />The plot is made interesting by the inclusion of a rabbit, which is clever but hardly profound. Many of the characters are heavily stereotyped -- the angry veterans, the terrified illegal aliens, the crooked cops, the indifferent feds, the bitchy tough lady station head, the crooked politician, the fat federale who looks like he was typecast as the Mexican in a Hollywood movie from the 1940s. All passably acted but again nothing special.<br /><br />I thought the main villains were pretty well done and fairly well acted. By the end of the movie you certainly knew who the good guys were and weren't. There was an emotional lift as the really bad ones got their just deserts. Very simplistic, but then you weren't expecting Hamlet, right? The only thing I found really annoying was the constant cuts to VDs daughter during the last fight scene.<br /><br />Not bad. Not good. Passable 4.

classifier prediction :  1
dataset label :  0


In the first example, the reviewer expresses mixed sentiments about the movie. While they mention enjoying these types of movies and praise Van Damme's performance, they also indicate that the movie is not as good as other Van Damme films and highlight its shortcomings. The classifier likely focused on the positive mentions of Van Damme and categorized it as a positive review, failing to consider the overall mixed sentiment expressed.

In the second example, the reviewer initially praises the director and highlights their past successful movies. However, they express disappointment with "The Shepherd," particularly with Jean-Claude Van Damme's performance, mentioning a lack of emotion and difficulty in empathizing with the hero. They also criticize the script and visual aspects. Despite mentioning some positive aspects like the action scenes and director's talent, the overall sentiment leans towards negativity. The classifier may have been influenced by the initial positive comments about the director and action sequences and failed to capture the negative sentiment towards the movie and Van Damme's performance.  
It is not helped by the final note given by the reviewer : 4 and passable, wich is difficult to either classify as positive or negative



### What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

Advantages of DistilBERT over Naive Bayes:

 - DistilBERT captures contextual information effectively, considering the surrounding words and their relationships.
 - It handles long-range dependencies in the text, capturing relationships between words that are far apart.
 - DistilBERT benefits from pre-training on a large corpus of text, allowing it to learn from extensive data and capture general language patterns.

Disadvantages of DistilBERT compared to Naive Bayes:

 - DistilBERT is computationally expensive and resource-intensive, requiring significant computational resources and memory.
 - It relies on large amounts of labeled data for fine-tuning, making it less suitable for tasks with limited labeled examples.

Advantages of DistilBERT over RNN/LSTM:

 - DistilBERT captures long-range dependencies more effectively than RNN or LSTM models.
 - It can process inputs in parallel, making it faster for batch processing and training.

Advantages of RNN/LSTM over DistilBERT:

 - RNN and LSTM models are designed for sequential data processing, making them suitable for tasks where the order of words or sequences is crucial.
 - They are generally less resource-intensive compared to transformer-based models like DistilBERT.